In [89]:
import numpy as np
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import cv2
from PIL import Image
%matplotlib inline

In [90]:
def create_model():
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(105, 105, 3))
    x = vgg_conv.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation='softmax')(x)
    
    for layer in vgg_conv.layers:
        layer.trainable=False
    
    model = Model(inputs=vgg_conv.input, outputs=x)
    
    return model

In [91]:
model = create_model()

In [92]:
#rmsprop = RMSprop(lr=0.0001)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [93]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    channel_shift_range=4,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [94]:
train_batchsize = 24
test_batchsize = 6

In [95]:
train_generator = train_datagen.flow_from_directory(
        './training_data',
        target_size=(105, 105),
        batch_size=train_batchsize,
        color_mode='rgb',
        class_mode='categorical')

Found 240 images belonging to 6 classes.


In [96]:
validation_generator = validation_datagen.flow_from_directory(
        './testing_data',
        target_size=(105, 105),
        batch_size=test_batchsize,
        class_mode='categorical',
        color_mode='rgb')

Found 60 images belonging to 6 classes.


In [97]:
checkpointer = ModelCheckpoint('cnn_model_3.hdf5',save_best_only=True)
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,callbacks=[checkpointer])

Epoch 1/50
10/10 [==============================] - 16s 2s/step - loss: 1.5936 - acc: 0.4125 - val_loss: 1.0629 - val_acc: 0.8000
Epoch 2/50
10/10 [==============================] - 14s 1s/step - loss: 1.0199 - acc: 0.6875 - val_loss: 0.7522 - val_acc: 0.8167
Epoch 3/50
10/10 [==============================] - 15s 1s/step - loss: 0.8000 - acc: 0.7250 - val_loss: 0.6910 - val_acc: 0.8500
Epoch 4/50
10/10 [==============================] - 15s 1s/step - loss: 0.6258 - acc: 0.7875 - val_loss: 0.6284 - val_acc: 0.8000
Epoch 5/50
10/10 [==============================] - 14s 1s/step - loss: 0.5815 - acc: 0.8292 - val_loss: 0.5409 - val_acc: 0.8333
Epoch 6/50
10/10 [==============================] - 15s 1s/step - loss: 0.5332 - acc: 0.8250 - val_loss: 0.5323 - val_acc: 0.8000
Epoch 7/50
10/10 [==============================] - 14s 1s/step - loss: 0.4768 - acc: 0.8625 - val_loss: 0.6168 - val_acc: 0.7500
Epoch 8/50
10/10 [==============================] - 15s 1s/step - loss: 0.4315 - acc: 0.87

In [98]:
model.load_weights('cnn_model_3.hdf5')

In [69]:
temp = cv2.imread('BankOfAmerica_header.png')
temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)

In [71]:
add = np.full((35,140,3),255,dtype=float)
crop = temp[130:200,480:620,:]
new = np.concatenate((add,crop,add))

In [74]:
new = cv2.resize(new, (105,105))
new = new/255

In [77]:
new = new.reshape((1,)+new.shape)

In [78]:
result = model.predict([new])

In [80]:
train_generator.class_indices

{'boa': 0,
 'capitalone': 1,
 'chase': 2,
 'citigroup': 3,
 'other': 4,
 'wellsfargo': 5}

array([9.9954033e-01, 1.5719052e-04, 3.3157190e-05, 3.7286100e-05,
       1.6437723e-04, 6.7666791e-05], dtype=float32)

In [79]:
result

array([[9.9954033e-01, 1.5719052e-04, 3.3157190e-05, 3.7286100e-05,
        1.6437723e-04, 6.7666791e-05]], dtype=float32)